In [1]:
import matplotlib.pyplot as plt  
import pandas as pd  
import numpy as np 
import seaborn as sns
import os, sys
from hsbmpy import get_file, define_labels, get_cluster_given_l, get_max_available_L
from geneontology import topic_analysis
import tensorflow as tf
from sklearn.metrics import homogeneity_completeness_v_measure
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import LatentDirichletAllocation
from lda import lda

/home/fvalle/anaconda3/envs/phd/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/fvalle/anaconda3/envs/phd/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/fvalle/anaconda3/envs/phd/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/fvalle/anaconda3/envs/phd/lib/python3.6/site-pack

In [2]:
os.chdir("/home/fvalle/phd/master_thesis")

In [3]:
level = 2
#setup = 'oversigma_10tissue'
#label = 'disease_type'
label='lda'
#label = 'uniq'
#labels = ['primary_site', 'disease_type']
#labels = ['primary_site', 'secondary_site']
directory=r"/home/fvalle/phd/datasets/breast_HDE"
#L=get_max_available_L(directory)
os.chdir(directory)

In [4]:
df = pd.read_csv("mainTable.csv", index_col=[0], header=[0]).dropna().astype(int)
totalobjcets = len(df.columns)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15991 entries, ENSG00000186160 to ENSG00000181458
Columns: 197 entries, 22c16f46-950f-4389-bb16-b5cec4d8b94e.FPKM.txt.gz to 67943973-e57a-4285-bd6e-995053d23fe0.FPKM.txt.gz
dtypes: int64(197)
memory usage: 24.2+ MB


In [5]:
df_files = pd.read_csv("files.dat", index_col=[0])
df_files.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1205 entries, fa9d8ef2-0fd0-4fd8-8266-e4760b77d103.FPKM.txt.gz to 110de223-52e1-43b6-8c69-399a0af8b174.FPKM.txt.gz
Data columns (total 6 columns):
cases.0.project.primary_site    1205 non-null object
cases.0.submitter_id            1205 non-null object
cases.0.project.disease_type    1205 non-null object
cancer.type                     1205 non-null object
Subtype_mRNA                    1205 non-null object
Subtype_Selected                1205 non-null object
dtypes: object(6)
memory usage: 65.9+ KB


In [ ]:
true_out = []
for sample in df.columns.values:
    try:
        true_out.append(get_file(sample, df_files)['Subtype_Selected'])
    except:
        print(sys.exc_info()[1])
        true_out.append('unknown')

In [ ]:
scores = {}

In [8]:
try:
    with open("clustersizes.txt",'r') as f:
        xl = np.array(f.read().split(sep='\n'))[:-1].astype(int)
except:
        xl=np.linspace(2,50,5, dtype=int)
xl

array([20,  2,  1])

In [9]:
df.T.values.shape

(197, 15991)

## hierarchical

In [ ]:
fig=plt.figure()
dend = shc.dendrogram(shc.linkage(df.T.values, method='ward'), leaf_rotation=90., leaf_font_size=8.,)
plt.xlabel("samples", fontsize=16)
plt.show()
fig.savefig("hierarchical_dendogram.pdf")

In [ ]:
#hierarchical
scores['hierarchical']={
    'h':[],
    'c':[],
    'V':[]
}
print("hierarchical")
os.system('mkdir -p hierarchical')
hierarchical_model = AgglomerativeClustering(n_clusters=1, affinity='euclidean', linkage='complete')  
for l,x in enumerate(xl):
    print("testing with %d clusters"%x)
    hierarchical_model.n_clusters=x
    out = hierarchical_model.fit_predict(df.T.values)
        
    #save clusters
    print("saving clusters")
    df_clusters = pd.DataFrame(index=np.arange(totalobjcets))
    for c in np.arange(out.max()+1)[::-1]:
        c_objects = df.columns[np.argwhere(out==c)].values.T[0]
        df_clusters.insert(0,"Cluster %d"%(c+1),np.concatenate((c_objects,[np.nan for _ in np.arange(totalobjcets-len(c_objects))])))
    df_clusters.dropna(axis=0,how='all', inplace=True)
    df_clusters.to_csv("hierarchical/hierarchical_level_%d_clusters.csv"%(l), index=False, header=True)
    
    score = (homogeneity_completeness_v_measure(true_out, out))
    scores['hierarchical']['h'].append(score[0])
    scores['hierarchical']['c'].append(score[1])
    scores['hierarchical']['V'].append(score[2])
    
pd.DataFrame(data=scores['hierarchical']).to_csv("hierarchical.scores", header=True, index=False)

## LDA

In [6]:
with open("topicsizes.txt",'r') as f:
    tl = np.array(f.read().split(sep='\n'))[:-1].astype(int)

In [7]:
print(xl, tl)

NameError: name 'xl' is not defined

In [ ]:
xl = xl[2:]
tl = xl
Sigmas = []

In [10]:
model=lda(n_jobs=1, verbose=2)
model.full_analysis(directory, xl,tl=None, label='Subtype_Selected')

model created
<class 'pandas.core.frame.DataFrame'>
Index: 15991 entries, ENSG00000186160 to ENSG00000181458
Columns: 197 entries, 22c16f46-950f-4389-bb16-b5cec4d8b94e.FPKM.txt.gz to 67943973-e57a-4285-bd6e-995053d23fe0.FPKM.txt.gz
dtypes: int64(197)
memory usage: 24.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
Index: 1205 entries, fa9d8ef2-0fd0-4fd8-8266-e4760b77d103.FPKM.txt.gz to 110de223-52e1-43b6-8c69-399a0af8b174.FPKM.txt.gz
Data columns (total 6 columns):
cases.0.project.primary_site    1205 non-null object
cases.0.submitter_id            1205 non-null object
cases.0.project.disease_type    1205 non-null object
cancer.type                     1205 non-null object
Subtype_mRNA                    1205 non-null object
Subtype_Selected                1205 non-null object
dtypes: object(6)
memory usage: 65.9+ KB
None
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
<class 'p

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s finished


iteration: 1 of max_iter: 5


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s finished


iteration: 2 of max_iter: 5


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


iteration: 5 of max_iter: 5


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


saving word-distr
saving topic-distr
saving clusters
saving metrics


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [ ]:
fig, ax=plt.subplots(figsize=(20,10))
for topic in df_D.columns:
    df_D[topic].hist(ax=ax, histtype='step', density=True, lw=2, label=topic)
ax.set_yscale('log')
ax.set_xlabel('$D^g[k]$', fontsize=20)
plt.tick_params(labelsize=20)
plt.tick_params(axis='x', rotation=60)
ax.legend(ncol=5)
plt.show()
fig.savefig("%s/distinctivness.pdf"%directory)

In [ ]:
topic_analysis(directory,1, 'lda')

In [ ]:
Sigmas

In [ ]:
homogeneity_completeness_v_measure(true_out, out)

## LDA Mallet

In [ ]:
from gensim.models.wrappers import LdaMallet
from gensim.corpora.dictionary import Dictionary
from gensim.test.utils import get_tmpfile
from gensim.corpora import MmCorpus

In [ ]:
corpus=list(map(list,map(lambda x: list(zip(range(len(df.index)), df[x])),df.columns)))
corpus

In [ ]:
dictionary=Dictionary([df.index])
[a for a in dictionary.items()]

In [ ]:
out_file=get_tmpfile("/home/fvalle/phd/datasets/tcga/oversampling_10tissue/corpus.mm")
MmCorpus.serialize(out_file, corpus, dictionary)

In [ ]:
model = LdaMallet("/home/fvalle/phd/Mallet/bin/mallet", workers=5, corpus=corpus, num_topics=15, id2word=dictionary)

In [ ]:
df_topics=pd.DataFrame(data=model.get_topics().T, index=[a[1] for a in dictionary.items()], columns=["Topic %d"%(t+1) for t in range(5)])
df_topics

In [ ]:
df_topic_distr=pd.read_csv(model.fdoctopics(), sep='\t', header=None, index_col=0).drop(1,1)
df_topic_distr.columns=["Topic %d"%(t+1) for t in range(5)]
df_topic_distr.index=df.columns
df_topic_distr

In [ ]:
df_topic_distr.apply(lambda x: x.idxmax().split(" ")[1], axis=1)

## hierachical on Altmann's output

In [ ]:
hiermodel = AgglomerativeClustering(n_clusters=10, affinity='euclidean', linkage='ward')

In [ ]:
with open('clustersizes.txt') as f:
    xl=np.array(f.read().split('\n')[:-1]).astype(int)

In [ ]:
os.system("mkdir -p hierhsbm")

In [ ]:
true_out = []
for sample in pd.read_csv("%s/%s_level_%d_topic-dist.csv"%('topsbm','topsbm',0), index_col=1).drop('i_doc', axis=1).index.values:
    try:
        true_out.append(get_file(sample, df_files)['primary_site'])
    except:
        print(sys.exc_info()[0])
        true_out.append('')

In [ ]:
scores['hierhsbm']={
    'h':[],
    'c':[],
    'V':[]
}
for l,n_clusters in enumerate(xl):
    print("Fitting level %d with %d clusters"%(l, n_clusters))
    df_topics = pd.read_csv("%s/%s_level_%d_topic-dist.csv"%('topsbm','topsbm',l), index_col=1).drop('i_doc', axis=1)
    df_clusters = pd.DataFrame(columns=["Cluster %d"%c for c in np.arange(n_clusters)+1])
    hiermodel.n_clusters=n_clusters
    out = hiermodel.fit_predict(df_topics.values)  
    for c in np.arange(out.max()+1)[::-1]:
        c_objects = df_topics.index[np.argwhere(out==c)].values.T[0]
        df_clusters["Cluster %d"%(c+1)]=np.concatenate((c_objects,[np.nan for _ in np.arange(len(df_topics.index)-len(c_objects))]))
    df_clusters.dropna(axis=0,how='all', inplace=True)
    df_clusters.to_csv("hierhsbm/hierhsbm_level_%d_clusters.csv"%(l), index=False, header=True)
    #metrics
    print("saving metrics")
    score = (homogeneity_completeness_v_measure(true_out, out))
    scores['hierhsbm']['h'].append(score[0])
    scores['hierhsbm']['c'].append(score[1])
    scores['hierhsbm']['V'].append(score[2])
    
pd.DataFrame(data=scores['hierhsbm']).to_csv("%s/hierhsbm.scores"%directory, header=True, index=False)